# IT ChatBot

### Import Packages

In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lmnop\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import numpy
import tflearn
import tensorflow
import random
import json
import pickle

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








### Data ingestion, evaluation and cleanup

In [3]:
with open('C:\\Users\\lmnop\\DockerFolder\\Tutorial\\intents3.json', encoding='utf-8') as file:
    data = json.load(file)

    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data['intents']:
        for pattern in intent['patterns']:
          wrds = nltk.word_tokenize(pattern)
          words.extend(wrds)
          docs_x.append(wrds)
          docs_y.append(intent['tag'])

        if intent['tag'] not in labels:
          labels.append(intent['tag'])
        
    words = [stemmer.stem(w.lower()) for w in words if w not in '?']
    words = sorted(list(set(words)))

    labels = sorted(labels)
    
    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]
    
    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)

    with open('data.pickle', 'wb') as f:
        pickle.dump((words, labels, training, output), f)

### Model Train and save

In [4]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 11999  | total loss: 0.00902 | time: 0.051s
| Adam | epoch: 1000 | loss: 0.00902 - acc: 0.9931 -- iter: 88/89
Training Step: 12000  | total loss: 0.00813 | time: 0.056s
| Adam | epoch: 1000 | loss: 0.00813 - acc: 0.9938 -- iter: 89/89
--
INFO:tensorflow:C:\Users\lmnop\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


### Model Evaluation

In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(w.lower()) for w in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

def chat():
    print('What?')
    while True:
        inp = input('You: ')
        if inp.lower() == 'quit':
            break
        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        if results[results_index] > .50:
            for tg in data['intents']:
                if tg['tag'] == tag:
                    responses = tg['responses']
            print(random.choice(responses))
        else:
            print('try again')
chat()

What?
You: hello
Hi there, how can I help?
You: what can you help with
Offering support for  1) Password Reset 2) Trouble-Shooting issues 3) Virus Issues 4) Printer issues 5) Other IT tasks
You: my printer is not printing
Check the printer is turned on, has paper, has ink/toner, paper in the correct tray etc. If its not an obvious issue, there may be a connection problem-the printer is probably not connected to the network properly, so call your IT Support helpdesk for help.
You: i can't read my usb
First things to check are: 1) Does the device work in a different USB port on the machine? 2) Are other devices recognised in that port? 3) Does the device work on another user’s machine?  If you have tried these troubleshooting methods and still no luck, then your IT support help-desk can proceed with some more in-depth troubleshooting.
